In [1]:
outp = "" #output directory

In [93]:
from pytube import YouTube
# misc
import os
import shutil
import math
import datetime
# plots
import matplotlib.pyplot as plt
%matplotlib inline
# image operation
import cv2


from PIL import Image
import numpy as np
from IPython.display import clear_output
import shutil

In [99]:
def process(im, aspect_ratio = 1., center_ratio = 1.):
    im=Image.fromarray(
      cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
     )
    imagex = int(im.size[0])
    imagey = int(im.size[1])
    width = min(imagex, imagey*aspect_ratio) * center_ratio
    height = min(imagex/aspect_ratio, imagey) * center_ratio
    left =(imagex - width)/2
    top = (imagey - height)/2
    box = (left,top,left+width,top+height)
    im = im.crop(box)
    im = im.resize((256,256), Image.ANTIALIAS)
    im=np.array(im)
    im=cv2.cvtColor(im, cv2.COLOR_RGB2BGR) 
    return im

def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[:1] == 'y':
            return True
        if reply[:1] == 'n':
            return False


#https://towardsdatascience.com/the-easiest-way-to-download-youtube-videos-using-python-2640958318ab
class FrameExtractor():
    '''
    Class used for extracting frames from a video file.
    '''
    def __init__(self, video_path):
        self.video_path = video_path
        self.vid_cap = cv2.VideoCapture(video_path)
        self.n_frames = int(self.vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = int(self.vid_cap.get(cv2.CAP_PROP_FPS))
        
    def get_video_duration(self):
        duration = self.n_frames/self.fps
        print(f'Duration: {datetime.timedelta(seconds=duration)}')
        
    def get_n_images(self, every_x_frame):
        n_images = math.floor(self.n_frames / every_x_frame) + 1
        print(f'Extracting every {every_x_frame} (nd/rd/th) frame would result in {n_images} images.')
        
    def extract_frames(self, every_x_frame=1, img_name="frame", max_frames=None, max_s=None,
                       dest_path=None, img_ext = '.jpg', startsec=0):
        if not self.vid_cap.isOpened():
            self.vid_cap = cv2.VideoCapture(self.video_path)
            
        self.vid_cap.set(cv2.CAP_PROP_POS_MSEC, startsec * 1000)
        
        if dest_path is None:
            dest_path = os.getcwd()
        else:
            if not os.path.isdir(dest_path):
                os.mkdir(dest_path)
                print(f'Created the following directory: {dest_path}')
        
        frame_cnt = 0
        img_cnt = 0
        
        firstframe=None
        lastframe=None

        while self.vid_cap.isOpened():
            
            success,image = self.vid_cap.read()
            
            if not success:
                break
             
            image = process(image)
            
            if frame_cnt % every_x_frame == 0:
                img_path = os.path.join(dest_path, ''.join([img_name, str(img_cnt), img_ext]))
                cv2.imwrite(img_path, image)  
                img_cnt += 1
            
            if max_frames is not None and img_cnt >= max_frames:
                break
            
            if max_s is not None and frame_cnt / self.fps >= max_s:
                break
            
            if img_cnt is 1:
                firstframe = image
            lastframe = image
                
            frame_cnt += 1
        
        f, axarr = plt.subplots(1,2)
        axarr[0].imshow(firstframe)
        axarr[1].imshow(lastframe)
        plt.show()
        if not yes_or_no("Does this look good?"):
            shutil.rmtree(dest_path)
        
        clear_output(wait=False)
        
        self.vid_cap.release()
        cv2.destroyAllWindows()

In [100]:
# video = YouTube('https://www.youtube.com/watch?v=kWxnadQI5Qw')
# vidinfo = video.streams.filter(type="video").get_highest_resolution()
# extension = vidinfo.mime_type.split('/')[-1]
# video.streams.get_by_itag(vidinfo.itag).download(output_path = outp, filename = "tempvideo")

fe = FrameExtractor(os.path.join(outp, "tempvideo." + extension))

dest_id = 0
while os.path.isdir(os.path.join(outp, str(dest_id))):
    dest_id += 1
dest_path = os.path.join(outp, str(dest_id))

fe.extract_frames(max_frames=60, dest_path=dest_path, startsec=34)

# os.remove(os.path.join(outp, "tempvideo." + extension))

In [ ]:
for video, 